# 1. 使用FUNCTION_CALL模式

In [ ]:
import os
import dotenv
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.chains.question_answering.map_reduce_prompt import system_template
from langchain_community.tools import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.tools import Tool

# 加载环境
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 获取tavily搜索工具的实例
search = TavilySearchResults(max_results=3)
# 获取对应工具的实例
search_tool = Tool(
	name="tavily_search",
	func=search.run,
	description="一个可以用来搜索互联网信息的工具。输入是一个搜索查询，输出是搜索结果的摘要。",
	return_direct=True
)
# 获取大模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# 提供提示词模板（以ChatPromptTemplate为例）
prompt_template = ChatPromptTemplate.from_messages(
	[
		("system", "你是一个有帮助的助手。可以回答用户的问题。如果你不知道答案，请使用tavily_search工具来搜索互联网信息。"),
		("human", "{input}"),
		("placeholder", "{agent_scratchpad}")
	]
)
# 获取Agent的实例：create_tool_calling_agent()
agent = create_tool_calling_agent(
	llm=llm,
	tools=[search_tool],
	prompt=prompt_template
)
# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
	agent=agent,
	tools=[search_tool],
	verbose=True
)
# 通过AgentExecutor调用invoke()方法,并得到响应
result = agent_executor.invoke({"input": "2025年最新AI技术趋势是什么？"})
# 处理响应
print(result)